# Quant Trading Strategies Final Project



In [32]:
import pandas as pd
import numpy as np
import requests 
import datetime as dt
import typing
from bs4 import BeautifulSoup
import statsmodels.formula.api as smf
import time
import quandl
import nasdaqdatalink as ndl
apikey = 'J_fXGeVW_zC6RaDeJSQv'
quandl.ApiConfig.api_key = apikey

In [2]:
trade_transactions = pd.read_csv('C:/Users/dcste/OneDrive/Economics_Research/Economics_Research/trade_transactions.csv').drop(columns=['Unnamed: 15','Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19'])
trade_transactions = trade_transactions[trade_transactions.type != 'Exchange']

In [3]:
trade_transactions

,transaction_date,disclosure_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,party,state,industry,sector
0,2/16/2023,2/16/2023,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Gary C. Peters,https://efdsearch.senate.gov/search/view/ptr/6...,NaN,NaN,NaN,NaN
1,1/23/2023,2/14/2023,Joint,GOLD,Barrick Gold Corporation Common Stock (BC),Stock,Purchase,"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
2,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sale (Partial),"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
3,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock <div class=...,Stock Option,Sale (Partial),"$1,001 - $15,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
4,1/18/2023,2/14/2023,Joint,CLF,Cleveland-Cliffs Inc. Common Stock,Stock,Purchase,"$100,001 - $250,000",--,Tommy Tuberville,https://efdsearch.senate.gov/search/view/ptr/9...,Republican,AL,Precious Metals,Basic Industries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8252,8/17/2012,8/17/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Sheldon Whitehouse,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8253,8/16/2012,8/16/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Pat Roberts,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8254,8/15/2012,8/15/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Rob Portman,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN
8255,8/2/2012,8/2/2012,NaN,NaN,This filing was disclosed via scanned PDF. Use...,PDF Disclosed Filing,NaN,Unknown,NaN,Thomas R. Carper,https://efdsearch.senate.gov/search/view/paper...,NaN,NaN,NaN,NaN


In [25]:
trade_transactions.type.unique()

array([nan, 'Purchase', 'Sale (Partial)', 'Sale (Full)'], dtype=object)

In [4]:
def filter_trade_type(trade_type:str):
    if trade_type == 'Purchase':
        trade_type = 'Buy'
    elif trade_type == 'Sale (Partial)':
        trade_type = 'Sell'
    elif trade_type == 'Sale (Full)':
        trade_type = 'Sell'
    return trade_type

In [5]:
trade_transactions['type'] = trade_transactions['type'].apply(filter_trade_type)

In [6]:
trade_transactions1  = trade_transactions[(trade_transactions.type == 'Buy')|(trade_transactions.type == 'Sell')]

In [15]:
trade_transactions['type'] = trade_transactions['type'].apply(filter_trade_type)
trade_transactions1 = trade_transactions1.dropna()

In [39]:
trade_transactions.type.unique()

array([nan, 'Buy', 'Sell'], dtype=object)

In [23]:
tickers = trade_transactions1.ticker.unique()

In [24]:
tickers

array(['GOLD', 'CLF', 'UAA', 'V', 'KO', 'AAPL', 'TXN', 'X', 'PXD', 'JPM',
       'CVS', 'BMY', 'BKNG', 'ZTS', 'AMZN', 'ALGN', 'RIO', 'MGM', 'AMT',
       'ATKR', 'HOLX', 'BHP', 'RMD', 'DLR', 'AXS', 'CME', 'BSM', 'EQNR',
       'MCD', 'RNWK', 'NU', 'DIS', 'PG', 'KMB', 'PYPL', 'CLX', 'QCOM',
       'C', 'FWONK', 'LSXMK', 'LBRDK', 'SYK', 'META', 'MDT', 'MLM',
       'FFIV', 'ELAN', 'CMCSA', 'BAX', 'GOOG', 'ADBE', 'RPM', 'NVDA',
       'MSFT', 'TJX', 'TMO', 'LRCX', 'TSM', 'INTC', 'F', 'ATVI', 'WIW',
       'LDP', 'BATRK', 'LBRDA', 'BATRA', 'XOM', 'CVX', 'VCTR', 'VRTX',
       'SCI', 'RELX', 'RH', 'PHM', 'GIS', 'GLPI', 'EXR', 'ETSY', 'ELS',
       'DQ', 'CROX', 'CSCO', 'AMGN', 'WIT', 'SEIC', 'TROW', 'NOC', 'MNST',
       'MASI', 'MKTX', 'IDXX', 'HON', 'GGG', 'IT', 'GRMN', 'FAST', 'EL',
       'CHD', 'AMAT', 'APD', 'MRO', 'OXY', 'FDX', 'VZ', 'MPW', 'ABT',
       'HQL', 'NID', 'FIVE', 'WFC', 'VTRS', 'ORCL', 'TSCO', 'SSYS', 'GWW',
       'COST', 'LMT', 'CAT', 'SBNY', 'MMM', 'CMS', 'IBM', 'SCCO

# Getting Tickers

In [33]:
start_date = '2012-07-25'
end_date = '2023-02-16'

In [30]:
prices = quandl.get_table('QUOTEMEDIA/PRICES',ticker= tickers,qopts = {'columns':['date','ticker','adj_close']}, date = {'gte':'2012-07-25', 'lte':'2023-02-16'}, paginate = True).set_index('date')


TypeError: Object of type ndarray is not JSON serializable

In [43]:
data = ndl.export_table('QUOTEMEDIA/PRICES', date={'gte':start_date,'lte':end_date },ticker=tickers,api_key = apikey)

TypeError: Object of type ndarray is not JSON serializable

In [ ]:
ndl.get_table()

In [26]:
prices[prices.ticker == tickers]

ValueError: ('Lengths must match to compare', (10000,), (679,))

In [28]:
prices.ticker.unique()

array(['ZZZ', 'ZZK', 'ZYXI', 'ZYNE', 'ZYME', 'ZY', 'ZXYZ_A', 'ZX', 'ZWS'],
      dtype=object)